In [16]:
import sqlite3
import pandas as pd

# Connect to SQLite database
database_path="../Datasets/database/longlist.db"
conn = sqlite3.connect(database_path)

# Create a cursor object
cursor = conn.cursor()

In [2]:
# Get database info: List all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(tables_query, conn)
print("Database Tables:\n", tables)

Database Tables:
           name
0      authors
1     authored
2        books
3   publishers
4      ratings
5  translators
6   translated


In [5]:
# Get column info for the 'episodes' table
table="authors"
columns_query = f"PRAGMA table_info({table});"
columns = pd.read_sql(columns_query, conn)
print("\nTable Schema for 'episodes':\n", columns)


Table Schema for 'episodes':
    cid     name     type  notnull dflt_value  pk
0    0       id  INTEGER        0       None   1
1    1     name     TEXT        0       None   0
2    2  country     TEXT        0       None   0
3    3    birth  INTEGER        0       None   0


In [6]:
# Query 1: Get the first 5 authors
df = pd.read_sql("SELECT * FROM authors LIMIT 5", conn)
print("\nFirst 5 authors:\n", df)


First 5 authors:
    id                 name    country  birth
0   1        Adania Shibli  Palestine   1974
1   2        Ahmed Saadawi       Iraq   1973
2   3  Alia Trabucco Zerán      Chile   1983
3   4      Amanda Svensson     Sweden   1987
4   5        Andrey Kurkov    Ukraine   1961


In [7]:
# Query 2: Count total authors
df_count = pd.read_sql("SELECT COUNT(*) AS total_authors FROM authors", conn)
print("\nTotal Authors:\n", df_count)


Total Authors:
    total_authors
0             72


In [12]:
# Query 3: Authors from Season 3
df_season3 = pd.read_sql("SELECT * FROM authors WHERE country = 'Spain' ORDER by birth DESC", conn)
print("\nAuthors from Spain 3:\n", df_season3)


Authors from Spain 3:
    id                  name country  birth
0  26       Gabriela Ybarra   Spain   1983
1  23          Eva Baltasar   Spain   1978
2  35         Javier Cercas   Spain   1962
3   8  Antonio Muñoz Molina   Spain   1956
4  21    Enrique Vila-Matas   Spain   1948


In [ ]:
# Close connection
conn.close()

### Loading from SQL file

In [20]:
cursor = conn.cursor()

In [21]:
query_path="Queries/"
query="longlist.sql"
qpath=query_path+query

# load query
with open(qpath, "r") as file:
    query=file.read()
    
# execute query
df = pd.read_sql(query, conn)
print("", df)

     id                publisher
0    1        And Other Stories
1    2             Charco Press
2    3          Europa Editions
3    4            Faber & Faber
4    5    Fitzcarraldo Editions
5    6             Granta Books
6    7           Harvill Secker
7    8             Honford Star
8    9            Jonathan Cape
9   10  Literatura Random House
10  11           Lolli Editions
11  12          MacLehose Press
12  13                 Oneworld
13  14            Peirene Press
14  15                  Picador
15  16         Portobello Books
16  17            Pushkin Press
17  18                 Riverrun
18  19          Sandstone Press
19  20                Scribe UK
20  21                 Scribner
21  22            Seagull Books
22  23           Serpent's Tail
23  24          Text Publishing
24  25            The New Press
25  26        Tilted Axis Press
26  27            Transit Books
27  28              Tuskar Rock
28  29            Verso Fiction
29  30                      W&N
30  31 

In [ ]:
# Query 3: Authors from Season 3
df_season3 = pd.read_sql("SELECT * FROM authors WHERE country = 'Spain' ORDER by birth DESC", conn)
print("\nAuthors from Spain 3:\n", df_season3)